<a href="https://colab.research.google.com/github/hubcborja/Proyecto_DataMining_NN/blob/main/Proyecto_Mineria_Redes_Neuronales_HuberBorja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción juegos de rugby

## Librerías

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
%matplotlib inline


In [2]:
# ! pip install bs4
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from html import unescape

## Primer acercamiento con la librería BeautifulSoup a un partido en específico

Documentación librería
https://beautiful-soup-4.readthedocs.io/en/latest/

Código tomado y modificados de los pasos de la página: https://www.datacamp.com/es/tutorial/web-scraping-using-python

In [ ]:
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
html = urlopen(url)


In [ ]:
soup = BeautifulSoup(html, 'lxml')
type(soup)


bs4.BeautifulSoup

In [ ]:
# Tener el título de la página
title = soup.title
print(title)


<title>Statistiques de Stade Toulousain - Section Paloise - J15 - 2025-2026 | Top 14 - Site Officiel</title>


In [ ]:
rows = soup.find_all('div')
print(rows)


[<div class="header__wrapper">
<sticky-header></sticky-header>
<div class="header__logo">
<a href="https://top14.lnr.fr" title="Retourner à l'accueil">
<img alt="TOP 14" class="header__logo-img" height="146" src="https://assets.lnr.fr/1/1/1/8/0/7/conversions/logo-top14.e32e7e9a-logo.webp"/>
</a>
</div>
<div class="header__content">
<div class="header-menu">
<div class="header-menu__part">
<div class="main-menu">
<main-menu :action='"https:\/\/top14.lnr.fr\/recherche"' :items='[{"type":"ticketing","name":"Billetterie","mobile_icon":null,"mobile_name":null,"device_type":"desktop","isExternal":true,"link":"https:\/\/billetterie.lnr.fr\/fr?utm_source=sitetop14&amp;utm_medium=CTA&amp;utm_campaign=organic","children":[]},{"type":"item","name":"Calendrier et r\u00e9sultats","mobile_icon":null,"mobile_name":null,"device_type":"mobile-and-desktop","isExternal":false,"link":"https:\/\/top14.lnr.fr\/calendrier-et-resultats","children":[]},{"type":"item","name":"Classement","mobile_icon":null,"mob

In [ ]:
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j7/11353-pau-toulouse/statistiques-du-match"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'es-ES,es;q=0.9',
    'Referer': 'https://top14.lnr.fr/'
}

response = requests.get(url, headers=headers)

print(f"Estado de la respuesta: {response.status_code}")
print(f"Longitud del contenido: {len(response.text)} caracteres")
print(f"Primeros 500 caracteres del HTML:\n{response.text[:500]}")


soup = BeautifulSoup(response.text, 'html.parser')
scripts = soup.find_all('script')
print(f"\nTotal de etiquetas <script> encontradas: {len(scripts)}")

Estado de la respuesta: 200
Longitud del contenido: 157846 caracteres
Primeros 500 caracteres del HTML:
<!DOCTYPE html>
<html lang="fr">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />
  <meta name="csrf-token" content="jkFpGD7Rs5j4xMUNrT3hS081PsgAbw7Le6GTpKW2" />

  <title>Statistiques de Section Paloise - Stade Toulousain - J7 - 2025-2026 | Top 14 - Site Officiel</title>
  <meta name="title" content="Statistiques de Section Paloise - Stade Toulousain - J7 - 2025-2026 | Top 14 - Site Officiel" />
  <meta 

Total de etiquetas <script> encontradas: 8


In [16]:
# <div class="match-header__season-day">
#                   Match terminé - J15 - 24/01/2026 - 21h00
#               </div>

#Si quiero extraer la fecha

url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
fecha = soup.find('div', class_='match-header__season-day')
fecha = fecha.get_text(strip=True) if fecha else "N/A"
fecha = "-".join(fecha.split("-")[2:])
fecha


' 24/01/2026 - 21h00'

## Extracción estadísticas del partido

In [14]:
# URL del partido
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

#Función para extraer estadísticas del partido
def extraer_resumen_partido(url):
    print(f" Extrayendo datos generales de: {url}...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Obtener el Resultado Final
    marcador_tag = soup.find('div', class_='title--large')
    marcador = marcador_tag.get_text(strip=True) if marcador_tag else "N/A"

    # Obtener fecha
    fecha = soup.find('div', class_='match-header__season-day')
    fecha = fecha.get_text(strip=True) if fecha else "N/A"
    fecha = "-".join(fecha.split("-")[2:])
    formato = " %d/%m/%Y - %Hh%M"
    fecha = datetime.strptime(fecha, formato)

    # Identificar Equipos
    equipo_local = soup.find_all('a', class_='match-header-club__title')[0].get_text(strip=True)
    equipo_visitante = soup.find_all('a', class_='match-header-club__title')[1].get_text(strip=True)

    print(f"Resultado Final: {equipo_local} {marcador} {equipo_visitante}, Fecha: {fecha}")

    # Extraer grupos de estadísticas (Points, Domination, Faltas, etc.)
    grupos = soup.find_all('div', class_='match-statistics__group')

    stats_totales = []

    for grupo in grupos:
        # Extraer el título de la categoría
        categoria = grupo.find('h2', class_='match-statistics__title')
        if not categoria: continue
        nombre_categoria = categoria.get_text(strip=True)

        # Extraer cada barra de estadística dentro del grupo
        filas = grupo.find_all('div', class_='stats-bar')
        for fila in filas:
            metrica = fila.find('div', class_='stats-bar__title').get_text(strip=True)
            valor_local = fila.find('div', class_='stats-bar__val--left').get_text(strip=True)
            valor_visitante = fila.find('div', class_='stats-bar__val--right').get_text(strip=True)

            stats_totales.append({
                'Categoría': nombre_categoria,
                'Métrica': metrica,
                equipo_local: valor_local,
                equipo_visitante: valor_visitante
            })

    return pd.DataFrame(stats_totales)


df_match_stats = extraer_resumen_partido(url)

if not df_match_stats.empty:
    print("\n Estadísticas:")
    display(df_match_stats)

    # Guardamos este resumen también
    df_match_stats.to_csv('resumen_global_partido.csv', index=False)
else:
    print("No se pudieron extraer las estadísticas grupales.")

 Extrayendo datos generales de: https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match...
Resultado Final: Stade Toulousain 59 - 22 Section Paloise, Fecha: 2026-01-24 21:00:00

 Estadísticas comparativas extraídas con éxito:


,Categoría,Métrica,Stade Toulousain,Section Paloise
0,Points,Essais accordés,9,3
1,Domination,Possession de la balle,57 %,43 %
2,Domination,Occupation,53 %,47 %
3,Domination,Possession dans son camp,43 %,57 %
4,Domination,Possession dans le camp adverse,48 %,52 %
5,Domination,Possession 22m adverses,50 %,50 %
6,Actions,Mêlées obtenues,8,6
7,Actions,Mêlées perdues,0,1
8,Actions,Mêlées gagnées,7,4
9,Actions,Mêlées refaites,1,1


## Extracción estadísticas de los jugadores

Código corregido con asistente Gemini

In [21]:
# URL del partido (Toulouse vs Pau)
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

#Función para extraer estadísticas de los jugadores del partido
def extraer_rugby_data(url):
    print(f"Iniciando extracción en: {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 1. Buscamos los componentes <players-ranking>
    # Hay dos: uno para el equipo local y otro para el visitante
    ranking_tags = soup.find_all('players-ranking')

    if len(ranking_tags) < 2:
        print("No se encontraron los datos de los jugadores. Verifica la conexión.")
        return None

    all_players = []

    # Nombres de los equipos
    equipo_local = soup.find_all('a', class_='match-header-club__title')[0].get_text(strip=True)
    equipo_visitante = soup.find_all('a', class_='match-header-club__title')[1].get_text(strip=True)
    equipos = [equipo_local, equipo_visitante]

    for idx, tag in enumerate(ranking_tags):
        raw_json = tag.get(':ranking')
        if raw_json:
            decoded_json = unescape(raw_json)
            data_list = json.loads(decoded_json)

            for p in data_list:
                all_players.append({
                    'Equipo': equipos[idx],
                    'Nombre': p['player']['name'],
                    'Posicion': p['position'],
                    'Minutos': int(p['minutesPlayed']),
                    'Puntos': int(p['nbPoints']),
                    'Essais': int(p['nbEssais']),
                    'Offload': int(p['offload']),
                    'Franchissement': int(p['lineBreak']),
                    'Tackles_Ok': int(p['totalSuccessfulTackles']),
                    'Tarjetas_Amarillas': int(p['nbCartonsJaunes'])
                })

    return pd.DataFrame(all_players)

# EJECUCIÓN
df_final = extraer_rugby_data(url)

if df_final is not None:
    print(f"Se cargaron {len(df_final)} jugadores.")
    display(df_final.sort_values(by='Puntos', ascending=False).head(10))


Iniciando extracción en: https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match
Se cargaron 46 jugadores.


,Equipo,Nombre,Posicion,Minutos,Puntos,Essais,Offload,Franchissement,Tackles_Ok,Tarjetas_Amarillas
18,Stade Toulousain,Thomas RAMOS,Demi d'ouverture,80,14,0,2,0,6,0
26,Section Paloise,F. BRAU-BOIRIE,Centre,80,10,2,0,1,10,0
31,Section Paloise,A. DESPERES RIGOU,Demi d'ouverture,68,7,0,0,0,8,0
4,Stade Toulousain,G. COLOMBE REAZEL,Centre,26,5,1,0,1,2,0
0,Stade Toulousain,Dorian ALDEGHERI,Pilier,54,5,1,0,1,7,0
2,Stade Toulousain,Joshua BRENNAN,Troisième ligne aile,47,5,1,1,0,8,0
14,Stade Toulousain,Julien MARCHAND,Talonneur,54,5,1,2,0,9,0
37,Section Paloise,Thomas LACLAYAT,Pilier,51,5,1,0,0,7,0
16,Stade Toulousain,E. MEAFOU,2ème ligne,63,5,1,1,0,8,0
22,Stade Toulousain,Jack WILLIS,Centre,33,5,1,0,2,8,0


## Extracción juegos temporadas


In [51]:
journee=['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14',
         'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26',
         'barrage', 'access-top-14', 'demi-finale', 'finale']
saisons=['2025-2026', '2024-2025', '2023-2024', '2022-2023', '2021-2022']

urls_journees=[]

#Creacion de los url a buscar para extraer
for s in saisons:
  for j in journee:
    url = f"https://top14.lnr.fr/calendrier-et-resultats/{s}/{j}"
    urls_journees.append(url)

urls_match=[]

for url in urls_journees:
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    link_match = [link.get('href') for link in soup.find_all('a', class_='match-line__score')]
    urls_match.append(link_match)
    time.sleep(5)

urls_planas = [match_url for jornada in urls_match for match_url in jornada]
urls_planas = [url for url in urls_planas if url]

separator="/"
urls_scarp=[]
for url in urls_planas:
    ult_s=separator.join([url, 'statistiques-du-match'])
    urls_scarp.append(ult_s)

urls_scarp_pd=pd.DataFrame(urls_scarp, columns=['url_match'])
urls_scarp_pd.to_csv('urls_scrapping_top14.csv', index=False)

[['https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11311-paris-montauban',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11313-perpignan-bayonne',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11308-castres-pau',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11309-lyon-racing-92',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11310-montpellier-toulon',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11312-bordeaux-begles-la-rochelle',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11307-clermont-toulouse'],
 ['https://top14.lnr.fr/feuille-de-match/2025-2026/j2/11320-montauban-lyon',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j2/11314-bayonne-montpellier',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j2/11317-pau-paris',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j2/11318-la-rochelle-clermont',
  'https://top14.lnr.fr/feuille-de-match/2025-2026/j2/11319-toulouse-perpignan',
  'https://top14.lnr.fr/feuille-de-match/202